In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
from pathlib import Path
module_path = str(Path.cwd().parents[0])
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
import time
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from benchmarking.data import dataset_loaders, DatasetGroup
from hnne.v2_plotter import format_time, format_count, plot_projection_grid, plot_v2_nested_2d, plot_layout_level_2d
from hnne.hierarchical_projection import multi_step_projection
from hnne.projector import HNNE

# Motivation
#### Goal: finalize and prepare for main branch merge (incl. updated docs, updated docstrings and example visulaization image for read me etc,.) by next week (07.10.25)

1. Guiding principal: auto fix start and end levels dependign on the data size and expected visual apparance
2. Impact on performance: depending on different start and end v2 levels
3. deep level start performance drop vs coarse level



## TODO Constantin

1. check if perfomance is about the same with new move logic vs the old one. params: v2=True, and  v1_behaviour=True vs v1_behaviour=False -- just run on some small or mediium dataset/s (if its similar then we remove the old move_points)
2. performance comparison of h-NNE vs h-NNE (v2)@ different values of start_cluster_view=None, v2_size_threshold=None (top 2 , top 3, deeper level start etc,.)
3. based on above, we can have a default settings for these params as a function of data size, see what insights can help us here.
4. find a very large dataset (order of more than > 10 or 20 million, wikimedia or any other wellknown), run and save finch output, then use few deeper levels satart and save projections. we can use this as example on repo and other places.

######## Note:  there is exisitng dataloader and datset set logic (see the the folder benchmarking)

## TODO Marios

1. prepare for main PR, update notebooks, update read me, update docs
2. add some new notebook/s demonstrating visualization choices with params:: prefered_num_clust=None, start_cluster_view=None, v2_size_threshold=None
3. Review the code and see if everythign works as expected with fit_transform anf transform etc.
4. any other things you think makes sense to check/include before going public

## TODO Saquib

1.  run some benchmarks along the lines as abbove on few datasets (sanity)
2.  re-evaluate the current v2 input params in HNNE init and remove those not necassary or user does not need to touch
3.  some cleanup in hierarchical_projection.py
4.  @ Marios/Constantin: please add any other thing here (that you think I need to do)?

### Dowload all hnne datasets as used in the paper as tar file (https://bwsyncandshare.kit.edu/s/Bp4gwZ4xcgj7otG). see the example loader below

In [43]:
data_path = Path(module_path, 'hnne_datasets')
## Load small, medium or large datasets, see ./hnne/benchmarking/data.py
loaders = dataset_loaders(dataset_group=DatasetGroup.small)
loaders.keys()

dict_keys(['coil_20', 'shuttle', 'mnist', 'fmnist'])

In [44]:
random_state=42
dataset_name = 'fmnist' #'cifar_10'
data, targets = loaders[dataset_name](data_path)